# pattern_scanner

> Scan Python code for replaceable CSS class patterns

In [ ]:
#| default_exp cli.pattern_scanner

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import ast
import re
from typing import List, Dict, Tuple, Optional, Set, Any
from dataclasses import dataclass

from cjm_fasthtml_tailwind.cli.cli_config import LibraryConfig, get_active_config
from cjm_fasthtml_tailwind.cli.example_discovery import get_example_pattern

## Data Structures

Define data structures for pattern scanning:

In [ ]:
#| export
@dataclass
class ClsPattern:
    """Represents a cls= pattern found in code."""
    line_number: int  # Line number where pattern was found
    full_expression: str  # The full cls=... expression
    css_classes: List[str]  # Individual CSS classes extracted
    context: str  # Code context around the pattern
    uses_combine_classes: bool  # Whether combine_classes is used

## AST Pattern Finder

Use Python's AST to find cls= patterns in code:

In [ ]:
#| export
class ClsPatternVisitor(ast.NodeVisitor):
    """AST visitor to find cls= patterns in Python code."""
    
    def __init__(
        self,
        source_lines: List[str]  # TODO: Add description
    ):
        """Initialize with source code lines for context extraction."""
        self.patterns: List[ClsPattern] = []
        self.source_lines = source_lines
    
    def visit_Call(
        self,
        node: ast.Call  # TODO: Add description
    ) -> None:  # TODO: Add return description
        """Visit function calls to find cls= keyword arguments."""
        # Check if this call has a cls keyword argument
        for keyword in node.keywords:
            if keyword.arg == 'cls':
                pattern = self._extract_pattern(keyword.value, node)
                if pattern:
                    self.patterns.append(pattern)
        
        # Continue visiting child nodes
        self.generic_visit(node)
    
    def _extract_pattern(
        self,
        value_node: ast.AST,  # TODO: Add description
        call_node: ast.Call  # TODO: Add description
    ) -> Optional[ClsPattern]:  # TODO: Add return description
        """Extract a ClsPattern from the cls= value node."""
        try:
            line_number = value_node.lineno
            
            # Get the full expression as a string
            full_expression = ast.unparse(value_node)
            
            # Check if combine_classes is used
            uses_combine_classes = isinstance(value_node, ast.Call) and \
                                 getattr(value_node.func, 'id', None) == 'combine_classes'
            
            # Extract CSS classes using the enhanced function
            css_classes = extract_css_classes_from_node(value_node)
            
            # Get context (the line containing the pattern)
            context_line = line_number - 1  # Convert to 0-based index
            if 0 <= context_line < len(self.source_lines):
                context = self.source_lines[context_line].strip()
            else:
                context = full_expression
            
            return ClsPattern(
                line_number=line_number,
                full_expression=full_expression,
                css_classes=css_classes,
                context=context,
                uses_combine_classes=uses_combine_classes
            )
        except Exception:
            # If we can't parse it, skip it
            return None

## Pattern Scanning Functions

Main functions to scan code for patterns:

In [ ]:
#| export
def scan_python_code(
    code: str  # Python source code as a string
) -> List[ClsPattern]:  # List of ClsPattern objects found in the code
    "Scan Python code for cls= patterns."
    try:
        # Parse the code into an AST
        tree = ast.parse(code)
        
        # Split code into lines for context extraction
        source_lines = code.splitlines()
        
        # Create and run the visitor
        visitor = ClsPatternVisitor(source_lines)
        visitor.visit(tree)
        
        return visitor.patterns
    except SyntaxError as e:
        print(f"Syntax error in code: {e}")
        return []
    except Exception as e:
        print(f"Error scanning code: {e}")
        return []

## Enhanced CSS Class Extraction

More sophisticated extraction handling different patterns:

In [ ]:
#| export
def extract_css_classes_from_node(
    node: ast.AST  # TODO: Add description
) -> List[str]:  # TODO: Add return description
    """
    Recursively extract CSS classes from an AST node.
    Handles various patterns including combine_classes calls.
    """
    classes = []
    
    if isinstance(node, ast.Constant) and isinstance(node.value, str):
        # Simple string literal
        classes.extend(node.value.split())
    
    elif isinstance(node, ast.Call):
        # Function call
        func_name = None
        if isinstance(node.func, ast.Name):
            func_name = node.func.id
        elif isinstance(node.func, ast.Attribute):
            # Handle chained calls like p.x(4)
            # For now, we'll skip these as they're already using the library
            return classes
        
        if func_name == 'combine_classes':
            # Extract from all arguments
            for arg in node.args:
                classes.extend(extract_css_classes_from_node(arg))
    
    elif isinstance(node, ast.List) or isinstance(node, ast.Tuple):
        # List or tuple of classes
        for elem in node.elts:
            classes.extend(extract_css_classes_from_node(elem))
    
    elif isinstance(node, ast.BinOp) and isinstance(node.op, ast.Add):
        # String concatenation: "flex " + "items-center"
        classes.extend(extract_css_classes_from_node(node.left))
        classes.extend(extract_css_classes_from_node(node.right))
    
    # Clean up extracted classes
    cleaned_classes = []
    for cls in classes:
        # Remove extra whitespace and filter empty strings
        cls = cls.strip()
        if cls and not cls.startswith('<'):
            cleaned_classes.append(cls)
    
    return cleaned_classes

## Testing Pattern Extraction

Let's test with the example code from your description:

In [ ]:
# Test with your example code
test_code = '''
from fasthtml.common import Div, Header, Nav, Main, Article, Aside, Footer, Img, Button, H1, H2, P

# Flexbox centered navigation
nav = Nav(
    Div("Logo", cls="font-bold"),
    Div(
        "Home", "About", "Contact",
        cls=combine_classes("flex", gap(4))
    ),
    Button("Sign In"),
    cls=combine_classes(
        "flex", 
        justify.between, 
        items.center, 
        "px-6", 
        "py-4", "items-center"
    )
)
'''

# Scan the code
patterns = scan_python_code(test_code)

# Display results
print(f"Found {len(patterns)} cls= patterns:\n")
for i, pattern in enumerate(patterns, 1):
    print(f"Pattern {i}:")
    print(f"  Line: {pattern.line_number}")
    print(f"  Expression: {pattern.full_expression}")
    print(f"  Uses combine_classes: {pattern.uses_combine_classes}")
    print(f"  CSS Classes: {pattern.css_classes}")
    print(f"  Context: {pattern.context}")
    print()

Found 3 cls= patterns:

Pattern 1:
  Line: 12
  Expression: combine_classes('flex', justify.between, items.center, 'px-6', 'py-4', 'items-center')
  Uses combine_classes: True
  CSS Classes: ['flex', 'px-6', 'py-4', 'items-center']
  Context: cls=combine_classes(

Pattern 2:
  Line: 6
  Expression: 'font-bold'
  Uses combine_classes: False
  CSS Classes: ['font-bold']
  Context: Div("Logo", cls="font-bold"),

Pattern 3:
  Line: 9
  Expression: combine_classes('flex', gap(4))
  Uses combine_classes: True
  CSS Classes: ['flex']
  Context: cls=combine_classes("flex", gap(4))



In [ ]:
# Test with various patterns
test_cases = '''
# Simple string
div1 = Div(cls="flex items-center justify-between")

# Multiple classes in one string
div2 = Div(cls="bg-blue-500 text-white px-4 py-2 rounded-lg")

# String concatenation
div3 = Div(cls="flex " + "items-center")

# Empty cls
div4 = Div(cls="")

# combine_classes with mixed content
div5 = Div(cls=combine_classes(
    "absolute",
    "top-0",
    p(4),  # This is already using the library
    "bg-white"
))
'''

patterns = scan_python_code(test_cases)
print(f"Found {len(patterns)} patterns in test cases:\n")

for i, pattern in enumerate(patterns, 1):
    print(f"Pattern {i}: Line {pattern.line_number}")
    print(f"  CSS Classes: {pattern.css_classes}")
    print(f"  Expression: {pattern.full_expression}")
    print()

Found 5 patterns in test cases:

Pattern 1: Line 3
  CSS Classes: ['flex', 'items-center', 'justify-between']
  Expression: 'flex items-center justify-between'

Pattern 2: Line 6
  CSS Classes: ['bg-blue-500', 'text-white', 'px-4', 'py-2', 'rounded-lg']
  Expression: 'bg-blue-500 text-white px-4 py-2 rounded-lg'

Pattern 3: Line 9
  CSS Classes: ['flex', 'items-center']
  Expression: 'flex ' + 'items-center'

Pattern 4: Line 12
  CSS Classes: []
  Expression: ''

Pattern 5: Line 15
  CSS Classes: ['absolute', 'top-0', 'bg-white']
  Expression: combine_classes('absolute', 'top-0', p(4), 'bg-white')



## Display Utilities

Functions to display scan results:

In [ ]:
#| export
def display_patterns(
    patterns: List[ClsPattern],  # List of ClsPattern objects to display
    show_context: bool = True  # Whether to show the code context
) -> None:  # TODO: Add return description
    "Display found patterns in a formatted way."
    if not patterns:
        print("No cls= patterns found in the code.")
        return
    
    print(f"Found {len(patterns)} cls= patterns:\n")
    
    for i, pattern in enumerate(patterns, 1):
        print(f"Pattern {i} (Line {pattern.line_number}):")
        
        if show_context:
            print(f"  Context: {pattern.context}")
        
        print(f"  Expression: {pattern.full_expression}")
        
        if pattern.uses_combine_classes:
            print(f"  ✓ Uses combine_classes")
        
        if pattern.css_classes:
            print(f"  CSS Classes ({len(pattern.css_classes)}):")
            for cls in pattern.css_classes:
                print(f"    - {cls}")
        else:
            print(f"  CSS Classes: (none)")
        
        print()  # Empty line between patterns

In [ ]:
#| export
def get_unique_css_classes(
    patterns: List[ClsPattern]  # List of ClsPattern objects
) -> Set[str]:  # Set of unique CSS class strings
    "Extract all unique CSS classes from a list of patterns."
    unique_classes = set()
    for pattern in patterns:
        unique_classes.update(pattern.css_classes)
    return unique_classes

In [ ]:
# Test the display function with the original example
patterns = scan_python_code(test_code)
display_patterns(patterns)

Found 3 cls= patterns:

Pattern 1 (Line 12):
  Context: cls=combine_classes(
  Expression: combine_classes('flex', justify.between, items.center, 'px-6', 'py-4', 'items-center')
  ✓ Uses combine_classes
  CSS Classes (4):
    - flex
    - px-6
    - py-4
    - items-center

Pattern 2 (Line 6):
  Context: Div("Logo", cls="font-bold"),
  Expression: 'font-bold'
  CSS Classes (1):
    - font-bold

Pattern 3 (Line 9):
  Context: cls=combine_classes("flex", gap(4))
  Expression: combine_classes('flex', gap(4))
  ✓ Uses combine_classes
  CSS Classes (1):
    - flex



In [ ]:
# Test extracting unique classes
all_patterns = scan_python_code(test_code) + scan_python_code(test_cases)
unique_classes = get_unique_css_classes(all_patterns)

print(f"Total unique CSS classes found: {len(unique_classes)}")
print("\nUnique classes:")
unique_classes

Total unique CSS classes found: 14

Unique classes:


{'absolute',
 'bg-blue-500',
 'bg-white',
 'flex',
 'font-bold',
 'items-center',
 'justify-between',
 'px-4',
 'px-6',
 'py-2',
 'py-4',
 'rounded-lg',
 'text-white',
 'top-0'}

## Assertion Pattern Extraction

Extract patterns from test assertion statements:

In [ ]:
#| export
import re
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

In [ ]:
#| export
@dataclass
class AssertionPattern:
    """Represents a pattern extracted from a test assertion."""
    css_class: str  # The CSS class string (e.g., "p-4")
    factory_expression: str  # The factory expression (e.g., "p(4)")
    module_name: str  # Module where this was found
    example_name: str  # Test function name

In [ ]:
#| export
def get_available_css_classes(
    assertion_patterns: List[AssertionPattern]  # List of assertion patterns from test examples
) -> Set[str]:  # Set of unique CSS class strings available in the library
    "Extract all unique CSS classes from assertion patterns. This handles multi-class assertion strings by splitting them."
    available_classes = set()
    
    for pattern in assertion_patterns:
        # Split multi-class strings and add each class
        css_classes = pattern.css_class.split()
        available_classes.update(css_classes)
    
    return available_classes

In [ ]:
#| export
def extract_assertion_patterns(
    source_code: str,  # Source code of the test function
    module_name: str,  # Name of the module containing the test
    example_name: str  # Name of the test function
) -> List[AssertionPattern]:  # List of AssertionPattern objects
    "Extract assertion patterns from test example source code."
    patterns = []
    
    # Split into lines for better processing
    lines = source_code.split('\n')
    
    for line in lines:
        # Skip empty lines and comments
        if not line.strip() or line.strip().startswith('#'):
            continue
            
        # Pattern 1: assert str(factory_expr) == "css-class"
        # Use simple greedy match that works
        match = re.search(r'assert\s+str\s*\(\s*(.+)\s*\)\s*==\s*["\']([^"\']+)["\']', line)
        if match:
            factory_expr = match.group(1).strip()
            css_class = match.group(2).strip()
            
            # Skip if factory_expr contains quotes (it's probably a string literal)
            if '"' not in factory_expr and "'" not in factory_expr:
                patterns.append(AssertionPattern(
                    css_class=css_class,
                    factory_expression=factory_expr,
                    module_name=module_name,
                    example_name=example_name
                ))
                continue
        
        # Pattern 2: assert factory_expr == "css-class" (without str())
        # Only if not already matched by pattern 1
        if 'str(' not in line:
            match = re.search(r'assert\s+(.+?)\s*==\s*["\']([^"\']+)["\']', line)
            if match:
                factory_expr = match.group(1).strip()
                css_class = match.group(2).strip()
                
                # Check if this looks like a factory expression
                if ('"' not in factory_expr and "'" not in factory_expr and 
                    ('(' in factory_expr or '.' in factory_expr)):
                    patterns.append(AssertionPattern(
                        css_class=css_class,
                        factory_expression=factory_expr,
                        module_name=module_name,
                        example_name=example_name
                    ))
    
    return patterns

In [ ]:
#| export
def collect_all_assertion_patterns(
) -> List[AssertionPattern]:  # List of AssertionPattern objects from all modules
    "Collect assertion patterns from all test examples in the library."
    from cjm_fasthtml_tailwind.cli.example_discovery import list_all_examples
    
    all_patterns = []
    
    # Get all test examples
    all_examples = list_all_examples()
    
    # Extract patterns from each example
    for module_name, examples in all_examples.items():
        for example in examples:
            patterns = extract_assertion_patterns(
                example.source,
                module_name,
                example.name
            )
            all_patterns.extend(patterns)
    
    return all_patterns

In [ ]:
assertion_patterns = collect_all_assertion_patterns()
assertion_classes = get_available_css_classes(assertion_patterns)
list(assertion_classes)[:20]

['snap-mandatory',
 'break-normal',
 '-hue-rotate-60',
 'shadow-zinc-500',
 'select-all',
 'transition-none',
 '-mask-linear-180',
 'inset-ring-green-500',
 'opacity-95',
 'col-span-full',
 'text-6xl',
 'decoration-yellow-500',
 'inset-ring-green-950',
 'drop-shadow-black',
 'row-start-2',
 'fill-white',
 'align-top',
 'rounded-s-lg',
 '-mask-linear-45',
 'p-0']

## Pattern Matching

Functions to match extracted CSS classes against available library classes:

In [ ]:
#| export
from enum import Enum

class MatchType(Enum):
    """Type of match found for a CSS class."""
    EXACT = "exact"  # Exact match found in available classes
    PATTERN = "pattern"  # Matches a known pattern (e.g., px-N)
    NO_MATCH = "no_match"  # No match found

@dataclass
class CSSClassMatch:
    """Represents a match result for a CSS class."""
    css_class: str  # The CSS class being matched
    match_type: MatchType  # Type of match found
    matched_pattern: Optional[str] = None  # The pattern it matches (for PATTERN type)
    similar_classes: List[str] = None  # Similar classes found in library
    suggested_replacement: Optional[str] = None  # Suggested replacement from library

In [ ]:
#| export
def tokenize_css_class(
    css_class: str  # CSS class string (e.g., "bg-blue-500" or "hover:text-white")
) -> List[str]:  # List of tokens (e.g., ["bg", "blue", "500"] or ["hover:text", "white"])
    "Tokenize a CSS class by splitting on hyphens. Handles modifiers (hover:, focus:, etc.) separately."
    # Handle modifiers (everything before the first colon)
    if ':' in css_class:
        # Split on the last colon to handle multiple modifiers
        parts = css_class.rsplit(':', 1)
        modifier = parts[0] + ':'
        base_class = parts[1]
        
        # Tokenize the base class part
        if base_class.startswith('-'):
            tokens = base_class[1:].split('-')
            if tokens:
                tokens[0] = '-' + tokens[0]
            return [modifier] + tokens
        else:
            return [modifier] + base_class.split('-')
    
    # Handle negative values (leading hyphen)
    if css_class.startswith('-'):
        # Keep the negative sign with the first token
        tokens = css_class[1:].split('-')
        if tokens:
            tokens[0] = '-' + tokens[0]
        return tokens
    else:
        return css_class.split('-')

In [ ]:
#| export
def find_pattern_matches(
    css_class: str,  # CSS class to match (e.g., "px-8" or "hover:text-white")
    available_classes: Set[str]  # Set of available CSS classes from the library
) -> Tuple[Optional[str], List[str]]:  # Tuple of (matched_pattern, similar_classes) - matched_pattern: Pattern prefix that matches (e.g., "px" for "px-8") - similar_classes: List of similar classes with the same pattern
    "Find pattern matches for a CSS class by progressively reducing tokens."
    tokens = tokenize_css_class(css_class)
    
    # Extract modifier if present
    modifier = ""
    base_tokens = tokens
    if tokens and tokens[0].endswith(':'):
        modifier = tokens[0]
        base_tokens = tokens[1:] if len(tokens) > 1 else []
    
    # Try progressively shorter patterns
    for i in range(len(base_tokens) - 1, 0, -1):
        # Build pattern prefix
        pattern_prefix = '-'.join(base_tokens[:i])
        
        # Include modifier in the search pattern if present
        if modifier:
            search_pattern = modifier + pattern_prefix + '-'
        else:
            search_pattern = pattern_prefix + '-'
        
        # Find all classes that start with this pattern
        similar_classes = [
            cls for cls in available_classes 
            if cls.startswith(search_pattern) and cls != css_class
        ]
        
        if similar_classes:
            # Return pattern without modifier for display
            return pattern_prefix, similar_classes
    
    return None, []

In [ ]:
#| export
def match_css_class(
    css_class: str,  # CSS class to match
    available_classes: Set[str]  # Set of available CSS classes from the library
) -> CSSClassMatch:  # CSSClassMatch object with match details
    "Match a CSS class against available library classes."
    # Check for exact match first
    if css_class in available_classes:
        return CSSClassMatch(
            css_class=css_class,
            match_type=MatchType.EXACT,
            suggested_replacement=css_class
        )
    
    # Try pattern matching
    pattern, similar_classes = find_pattern_matches(css_class, available_classes)
    
    if pattern and similar_classes:
        # Sort similar classes for consistent output
        similar_classes = sorted(similar_classes)[:5]  # Limit to 5 examples
        
        return CSSClassMatch(
            css_class=css_class,
            match_type=MatchType.PATTERN,
            matched_pattern=pattern,
            similar_classes=similar_classes,
            suggested_replacement=None  # Could be enhanced to suggest closest match
        )
    
    # No match found
    return CSSClassMatch(
        css_class=css_class,
        match_type=MatchType.NO_MATCH
    )

In [ ]:
#| export
def match_css_classes(
    css_classes: List[str],  # List of CSS classes to match
    available_classes: Set[str]  # Set of available CSS classes from the library
) -> Dict[str, CSSClassMatch]:  # Dictionary mapping CSS classes to their match results
    "Match multiple CSS classes against available library classes."
    matches = {}
    for css_class in css_classes:
        matches[css_class] = match_css_class(css_class, available_classes)
    return matches

In [ ]:
#| export
def display_match_results(
    matches: Dict[str, CSSClassMatch]  # Dictionary of CSS classes to their match results
) -> None:  # TODO: Add return description
    "Display match results in a formatted way."
    # Group by match type
    exact_matches = []
    pattern_matches = []
    no_matches = []
    
    for css_class, match in matches.items():
        if match.match_type == MatchType.EXACT:
            exact_matches.append(css_class)
        elif match.match_type == MatchType.PATTERN:
            pattern_matches.append((css_class, match))
        else:
            no_matches.append(css_class)
    
    # Display results
    print("CSS Class Analysis Results:")
    print("=" * 60)
    
    if exact_matches:
        print(f"\n✓ Exact Matches ({len(exact_matches)}):")
        for cls in sorted(exact_matches):
            print(f"  - {cls}")
    
    if pattern_matches:
        print(f"\n~ Pattern Matches ({len(pattern_matches)}):")
        for cls, match in sorted(pattern_matches):
            print(f"  - {cls} → matches pattern '{match.matched_pattern}-*'")
            if match.similar_classes:
                print(f"    Examples: {', '.join(match.similar_classes[:3])}")
    
    if no_matches:
        print(f"\n✗ No Matches ({len(no_matches)}):")
        for cls in sorted(no_matches):
            print(f"  - {cls}")
    
    # Summary
    total = len(matches)
    replaceable = len(exact_matches) + len(pattern_matches)
    print(f"\nSummary: {replaceable}/{total} classes are potentially replaceable")

In [ ]:
#| export
def analyze_code_patterns(
    code: str  # Python source code to analyze
) -> Dict[str, Any]:  # Dictionary with analysis results including patterns found and suggestions
    "Analyze Python code for replaceable CSS patterns."
    # Scan for patterns
    patterns = scan_python_code(code)
    
    # Get unique CSS classes
    unique_classes = get_unique_css_classes(patterns)
    
    # Get available classes from library
    assertion_patterns = collect_all_assertion_patterns()
    available_classes = get_available_css_classes(assertion_patterns)
    
    # Match CSS classes
    matches = match_css_classes(list(unique_classes), available_classes)
    
    # Prepare results
    results = {
        'total_patterns': len(patterns),
        'unique_classes': len(unique_classes),
        'matches': matches,
        'summary': {
            'exact_matches': sum(1 for m in matches.values() if m.match_type == MatchType.EXACT),
            'pattern_matches': sum(1 for m in matches.values() if m.match_type == MatchType.PATTERN),
            'no_matches': sum(1 for m in matches.values() if m.match_type == MatchType.NO_MATCH),
        },
        'patterns': patterns
    }
    
    return results

In [ ]:
#| export
def display_code_analysis(
    code: str  # Python source code to analyze
) -> None:  # TODO: Add return description
    "Analyze and display replaceable patterns in Python code."
    results = analyze_code_patterns(code)
    
    print("Code Analysis Report")
    print("=" * 60)
    print(f"Total cls= patterns found: {results['total_patterns']}")
    print(f"Unique CSS classes: {results['unique_classes']}")
    print()
    
    # Display summary
    summary = results['summary']
    total_replaceable = summary['exact_matches'] + summary['pattern_matches']
    print(f"Replaceable Classes: {total_replaceable}/{results['unique_classes']}")
    print(f"  - Exact matches: {summary['exact_matches']}")
    print(f"  - Pattern matches: {summary['pattern_matches']}")
    print(f"  - No matches: {summary['no_matches']}")
    print()
    
    # Display matches by type
    display_match_results(results['matches'])
    
    # Show patterns with line numbers
    if results['patterns']:
        print("\nPatterns by Line:")
        print("-" * 60)
        for pattern in results['patterns']:
            replaceable = "✓" if pattern.css_classes and any(
                results['matches'].get(cls, CSSClassMatch(cls, MatchType.NO_MATCH)).match_type != MatchType.NO_MATCH 
                for cls in pattern.css_classes
            ) else "✗"
            
            print(f"Line {pattern.line_number}: {replaceable} {pattern.full_expression}")
            if pattern.uses_combine_classes:
                print(f"         ↳ Already uses combine_classes")

## Migration Suggestions

Functions to provide migration suggestions based on test examples:

In [ ]:
#| export
def find_assertion_for_class(
    css_class: str,  # The CSS class to find (e.g., "px-6")
    assertion_patterns: List[AssertionPattern]  # List of all assertion patterns from tests
) -> Optional[AssertionPattern]:  # AssertionPattern if found, None otherwise
    "Find the assertion pattern that demonstrates how to use a specific CSS class. Prioritizes exact single-class matches over multi-class assertions."
    # First pass: look for exact single-class matches
    for pattern in assertion_patterns:
        if pattern.css_class == css_class:
            return pattern
    
    # Second pass: look in multi-class assertions
    for pattern in assertion_patterns:
        assertion_classes = pattern.css_class.split()
        if css_class in assertion_classes and len(assertion_classes) > 1:
            return pattern
    
    return None

In [ ]:
#| export
def find_pattern_examples(
    pattern_prefix: str,  # Pattern prefix to match (e.g., "px" for px-* pattern)
    assertion_patterns: List[AssertionPattern]  # List of all assertion patterns from tests
) -> List[AssertionPattern]:  # List of AssertionPattern objects that match the pattern
    "Find assertion examples that match a pattern prefix."
    matching_patterns = []
    pattern_with_dash = pattern_prefix + '-'
    
    for pattern in assertion_patterns:
        # Check each class in the assertion
        assertion_classes = pattern.css_class.split()
        for cls in assertion_classes:
            if cls.startswith(pattern_with_dash):
                matching_patterns.append(pattern)
                break  # Only add once per assertion
                
    return matching_patterns

In [ ]:
#| export
def get_migration_suggestions(
    matches: Dict[str, CSSClassMatch],  # Dictionary of CSS class matches
    assertion_patterns: List[AssertionPattern],  # List of all assertion patterns from tests
    config: Optional[LibraryConfig] = None  # Optional configuration
) -> Dict[str, List[str]]:  # Dictionary mapping CSS classes to their migration suggestions
    "Generate migration suggestions for matched CSS classes."
    if config is None:
        config = get_active_config()
    
    suggestions = {}
    
    for css_class, match in matches.items():
        class_suggestions = []
        
        if match.match_type == MatchType.EXACT:
            # Find the exact assertion that demonstrates this class
            assertion = find_assertion_for_class(css_class, assertion_patterns)
            if assertion:
                # Extract feature name from example name
                feature = get_example_pattern(assertion.module_name).match(assertion.example_name).group(1)
                suggestion = f"View example: {config.cli_command} example {assertion.module_name} {feature}"
                class_suggestions.append(suggestion)
                
                # Check if the factory expression might be a helper function
                # Helper functions typically have parentheses and might be in the feature name
                if '(' in assertion.factory_expression and ')' in assertion.factory_expression:
                    # Extract potential function name
                    func_name = assertion.factory_expression.split('(')[0].strip()
                    
                    # Check if this might be a helper (not a factory like p.x or w.full)
                    if not ('.' in func_name and len(func_name.split('.')) == 2):
                        # This looks like a helper function
                        helper_suggestion = f"View helper: {config.cli_command} helper {assertion.module_name} {func_name}"
                        class_suggestions.append(helper_suggestion)
                
        elif match.match_type == MatchType.PATTERN:
            # Find examples that demonstrate this pattern
            pattern_assertions = find_pattern_examples(match.matched_pattern, assertion_patterns)
            if pattern_assertions:
                # Use a set to track unique suggestions
                unique_suggestions = set()
                
                # Show up to 3 unique examples
                for assertion in pattern_assertions:
                    feature = get_example_pattern(assertion.module_name).match(assertion.example_name).group(1)
                    suggestion = f"{config.cli_command} example {assertion.module_name} {feature}"
                    unique_suggestions.add(suggestion)
                    
                    # Stop if we have 3 unique suggestions
                    if len(unique_suggestions) >= 3:
                        break
                
                # Convert set back to list with proper formatting
                for i, suggestion in enumerate(sorted(unique_suggestions), 1):
                    class_suggestions.append(f"Pattern example {i}: {suggestion}")
        
        if class_suggestions:
            suggestions[css_class] = class_suggestions
    
    return suggestions

In [ ]:
#| export
def display_migration_suggestions(
    code: str  # Python source code to analyze
) -> None:  # TODO: Add return description
    "Analyze code and display migration suggestions."
    # Get analysis results
    results = analyze_code_patterns(code)
    
    # Get assertion patterns
    assertion_patterns = collect_all_assertion_patterns()
    
    # Get migration suggestions
    suggestions = get_migration_suggestions(results['matches'], assertion_patterns)
    
    if not suggestions:
        print("No migration suggestions available.")
        return
    
    print("Migration Suggestions")
    print("=" * 60)
    print()
    
    for css_class in sorted(suggestions.keys()):
        match = results['matches'][css_class]
        match_type = "✓" if match.match_type == MatchType.EXACT else "~"
        
        print(f"{match_type} {css_class}:")
        for suggestion in suggestions[css_class]:
            print(f"  → {suggestion}")
        print()

In [ ]:
#| export
def analyze_and_suggest(
    code: str  # Python source code to analyze
) -> None:  # TODO: Add return description
    "Perform complete analysis of code with migration suggestions."
    # First show the analysis
    display_code_analysis(code)
    
    # Then show migration suggestions if any replaceable patterns found
    results = analyze_code_patterns(code)
    summary = results['summary']
    
    if summary['exact_matches'] > 0 or summary['pattern_matches'] > 0:
        print("\n")
        display_migration_suggestions(code)

## Testing Pattern Matching

Test the matching logic with various CSS classes:

In [ ]:
# Test updated tokenization
print("Testing updated tokenization:")
test_classes = [
    "flex", 
    "px-6", 
    "bg-blue-500", 
    "-mt-4", 
    "hover:text-white",
    "lg:grid-cols-4",
    "focus:ring-2",
    "hover:bg-blue-600"
]
for cls in test_classes:
    tokens = tokenize_css_class(cls)
    print(f"  {cls:<20} → {tokens}")

Testing updated tokenization:
  flex                 → ['flex']
  px-6                 → ['px', '6']
  bg-blue-500          → ['bg', 'blue', '500']
  -mt-4                → ['-mt', '4']
  hover:text-white     → ['hover:', 'text', 'white']
  lg:grid-cols-4       → ['lg:', 'grid', 'cols', '4']
  focus:ring-2         → ['focus:', 'ring', '2']
  hover:bg-blue-600    → ['hover:', 'bg', 'blue', '600']


In [ ]:
# Get available classes from assertions
assertion_patterns = collect_all_assertion_patterns()
available_classes = get_available_css_classes(assertion_patterns)

# Test with the example from the original test code
test_css_classes = ["flex", "font-bold", "items-center", "px-6", "py-4"]

print("\nTesting CSS class matching:")
print("=" * 60)

for css_class in test_css_classes:
    match_result = match_css_class(css_class, available_classes)
    
    print(f"\nClass: '{css_class}'")
    print(f"  Match Type: {match_result.match_type.value}")
    
    if match_result.match_type == MatchType.EXACT:
        print(f"  ✓ Exact match found")
    elif match_result.match_type == MatchType.PATTERN:
        print(f"  ~ Pattern match: '{match_result.matched_pattern}-*'")
        print(f"  Similar classes: {match_result.similar_classes[:3]}...")
    else:
        print(f"  ✗ No match found")


Testing CSS class matching:

Class: 'flex'
  Match Type: exact
  ✓ Exact match found

Class: 'font-bold'
  Match Type: exact
  ✓ Exact match found

Class: 'items-center'
  Match Type: exact
  ✓ Exact match found

Class: 'px-6'
  Match Type: exact
  ✓ Exact match found

Class: 'py-4'
  Match Type: exact
  ✓ Exact match found


In [ ]:
# Test with more diverse CSS classes
print("\nTesting with additional CSS classes:")
print("=" * 60)

additional_test_classes = [
    # Should be exact matches
    "block", "absolute", "flex",
    # Should be pattern matches
    "px-8", "py-12", "mt-16", "gap-10", 
    # Should be no match
    "font-bold", "text-blue-600", "hover:bg-gray-100",
    # Edge cases
    "-mx-4", "w-1/3", "lg:grid-cols-4"
]

for css_class in additional_test_classes:
    match_result = match_css_class(css_class, available_classes)
    
    status = "✓" if match_result.match_type == MatchType.EXACT else \
             "~" if match_result.match_type == MatchType.PATTERN else "✗"
    
    print(f"{status} {css_class:<20} → {match_result.match_type.value:<10}", end="")
    
    if match_result.matched_pattern:
        print(f" (pattern: {match_result.matched_pattern})")
    else:
        print()


Testing with additional CSS classes:
✓ block                → exact     
✓ absolute             → exact     
✓ flex                 → exact     
✓ px-8                 → exact     
~ py-12                → pattern    (pattern: py)
~ mt-16                → pattern    (pattern: mt)
~ gap-10               → pattern    (pattern: gap)
✓ font-bold            → exact     
~ text-blue-600        → pattern    (pattern: text-blue)
✗ hover:bg-gray-100    → no_match  
~ -mx-4                → pattern    (pattern: -mx)
~ w-1/3                → pattern    (pattern: w)
✓ lg:grid-cols-4       → exact     


In [ ]:
# Test batch matching with the original example
print("\nBatch matching for original example:")
print("=" * 60)

# Extract unique CSS classes from the test code
patterns = scan_python_code(test_code)
unique_classes_in_code = get_unique_css_classes(patterns)

# Match all classes
matches = match_css_classes(list(unique_classes_in_code), available_classes)

# Display results
display_match_results(matches)


Batch matching for original example:
CSS Class Analysis Results:

✓ Exact Matches (5):
  - flex
  - font-bold
  - items-center
  - px-6
  - py-4

Summary: 5/5 classes are potentially replaceable


In [ ]:
# Test complete code analysis
print("Complete Code Analysis:")
print()
display_code_analysis(test_code)

Complete Code Analysis:

Code Analysis Report
Total cls= patterns found: 3
Unique CSS classes: 5

Replaceable Classes: 5/5
  - Exact matches: 5
  - Pattern matches: 0
  - No matches: 0

CSS Class Analysis Results:

✓ Exact Matches (5):
  - flex
  - font-bold
  - items-center
  - px-6
  - py-4

Summary: 5/5 classes are potentially replaceable

Patterns by Line:
------------------------------------------------------------
Line 12: ✓ combine_classes('flex', justify.between, items.center, 'px-6', 'py-4', 'items-center')
         ↳ Already uses combine_classes
Line 6: ✓ 'font-bold'
Line 9: ✓ combine_classes('flex', gap(4))
         ↳ Already uses combine_classes


In [ ]:
# Test with a more complex example
complex_example = '''
from fasthtml.common import Div, Button, Section

# Hero section with various utilities
hero = Section(
    Div(
        "Welcome to our site",
        cls="text-6xl font-bold text-gray-900 mb-4"
    ),
    Div(
        "Build amazing things with FastHTML",
        cls="text-xl text-gray-600 mb-8"
    ),
    Button(
        "Get Started",
        cls="px-8 py-3 bg-blue-600 text-white rounded-lg hover:bg-blue-700"
    ),
    cls="flex flex-col items-center justify-center min-h-screen px-4"
)
'''

print("\n\nComplex Example Analysis:")
print()
display_code_analysis(complex_example)



Complex Example Analysis:

Code Analysis Report
Total cls= patterns found: 4
Unique CSS classes: 19

Replaceable Classes: 18/19
  - Exact matches: 12
  - Pattern matches: 6
  - No matches: 1

CSS Class Analysis Results:

✓ Exact Matches (12):
  - flex
  - flex-col
  - font-bold
  - items-center
  - justify-center
  - mb-4
  - min-h-screen
  - px-8
  - rounded-lg
  - text-6xl
  - text-white
  - text-xl

~ Pattern Matches (6):
  - bg-blue-600 → matches pattern 'bg-blue-*'
    Examples: bg-blue-300, bg-blue-300/75, bg-blue-500
  - mb-8 → matches pattern 'mb-*'
    Examples: mb-4
  - px-4 → matches pattern 'px-*'
    Examples: px-6, px-8
  - py-3 → matches pattern 'py-*'
    Examples: py-4, py-8
  - text-gray-600 → matches pattern 'text-gray-*'
    Examples: text-gray-500
  - text-gray-900 → matches pattern 'text-gray-*'
    Examples: text-gray-500

✗ No Matches (1):
  - hover:bg-blue-700

Summary: 18/19 classes are potentially replaceable

Patterns by Line:
-----------------------------

In [ ]:
# Test migration suggestions with original example
print("Migration Suggestions for Original Example:")
print()
display_migration_suggestions(test_code)

Migration Suggestions for Original Example:

Migration Suggestions

✓ flex:
  → View example: cjm-tailwind-explore example flexbox_and_grid display

✓ font-bold:
  → View example: cjm-tailwind-explore example typography font_weight

✓ items-center:
  → View example: cjm-tailwind-explore example flexbox_and_grid align

✓ px-6:
  → View example: cjm-tailwind-explore example spacing helper
  → View helper: cjm-tailwind-explore helper spacing pad

✓ py-4:
  → View example: cjm-tailwind-explore example spacing helper
  → View helper: cjm-tailwind-explore helper spacing pad



In [ ]:
# Test with complex example
print("\n\nMigration Suggestions for Complex Example:")
print()
display_migration_suggestions(complex_example)



Migration Suggestions for Complex Example:

Migration Suggestions

~ bg-blue-600:
  → Pattern example 1: cjm-tailwind-explore example backgrounds color
  → Pattern example 2: cjm-tailwind-explore example backgrounds opacity

✓ flex:
  → View example: cjm-tailwind-explore example flexbox_and_grid display

✓ flex-col:
  → View example: cjm-tailwind-explore example flexbox_and_grid direction

✓ font-bold:
  → View example: cjm-tailwind-explore example typography font_weight

✓ items-center:
  → View example: cjm-tailwind-explore example flexbox_and_grid align

✓ justify-center:
  → View example: cjm-tailwind-explore example flexbox_and_grid justify

✓ mb-4:
  → View example: cjm-tailwind-explore example spacing margin_directional

~ mb-8:
  → Pattern example 1: cjm-tailwind-explore example spacing margin_directional

✓ min-h-screen:
  → View example: cjm-tailwind-explore example sizing min_height

~ px-4:
  → Pattern example 1: cjm-tailwind-explore example spacing directional
  → Patter

In [ ]:
# Test analysis and migration suggestions with original example
print("Analysis & Migration Suggestions for Original Example:")
print()
analyze_and_suggest(test_code)

Analysis & Migration Suggestions for Original Example:

Code Analysis Report
Total cls= patterns found: 3
Unique CSS classes: 5

Replaceable Classes: 5/5
  - Exact matches: 5
  - Pattern matches: 0
  - No matches: 0

CSS Class Analysis Results:

✓ Exact Matches (5):
  - flex
  - font-bold
  - items-center
  - px-6
  - py-4

Summary: 5/5 classes are potentially replaceable

Patterns by Line:
------------------------------------------------------------
Line 12: ✓ combine_classes('flex', justify.between, items.center, 'px-6', 'py-4', 'items-center')
         ↳ Already uses combine_classes
Line 6: ✓ 'font-bold'
Line 9: ✓ combine_classes('flex', gap(4))
         ↳ Already uses combine_classes


Migration Suggestions

✓ flex:
  → View example: cjm-tailwind-explore example flexbox_and_grid display

✓ font-bold:
  → View example: cjm-tailwind-explore example typography font_weight

✓ items-center:
  → View example: cjm-tailwind-explore example flexbox_and_grid align

✓ px-6:
  → View example: 

## File Input Support

Functions to scan Python files and Jupyter notebooks:

In [ ]:
#| export
def scan_python_file(
    file_path: str  # Path to the Python file
) -> List[ClsPattern]:  # List of ClsPattern objects found in the file
    "Scan a Python file for cls= patterns."
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            code = f.read()
        
        return scan_python_code(code)
    
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return []
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

In [ ]:
#| export
def scan_jupyter_notebook(
    notebook_path: str  # Path to the Jupyter notebook (.ipynb)
) -> List[ClsPattern]:  # List of ClsPattern objects found in the notebook
    "Scan a Jupyter notebook for cls= patterns."
    try:
        from execnb.nbio import read_nb
        
        # Read the notebook
        nb = read_nb(notebook_path)
        
        all_patterns = []
        
        # Iterate through cells
        for cell in nb.cells:
            # Only process code cells
            if cell.cell_type == 'code':
                # Get the source code from the cell
                source = cell.source
                if isinstance(source, list):
                    source = ''.join(source)
                
                # Skip empty cells
                if not source.strip():
                    continue
                
                # Scan the cell code
                patterns = scan_python_code(source)
                
                # Add cell metadata to help identify location
                for pattern in patterns:
                    # Adjust line numbers to be relative to the cell
                    pattern.context = f"[Cell {cell.idx_}] {pattern.context}"
                
                all_patterns.extend(patterns)
        
        return all_patterns
        
    except ImportError:
        print("Error: execnb not installed. Install with: pip install execnb")
        return []
    except FileNotFoundError:
        print(f"Error: Notebook not found: {notebook_path}")
        return []
    except Exception as e:
        print(f"Error reading notebook {notebook_path}: {e}")
        return []

In [ ]:
#| export
import os
from enum import Enum

class InputType(Enum):
    """Type of input being scanned."""
    CODE = "code"  # Direct code string
    PYTHON_FILE = "python_file"  # .py file
    NOTEBOOK = "notebook"  # .ipynb file
    
def detect_input_type(
    input_source: str  # Code string or file path
) -> InputType:  # InputType enum value
    "Detect the type of input based on the source string."
    # Check if it's a file path
    if os.path.exists(input_source):
        if input_source.endswith('.py'):
            return InputType.PYTHON_FILE
        elif input_source.endswith('.ipynb'):
            return InputType.NOTEBOOK
    
    # Check if it looks like a file path (even if file doesn't exist)
    if '/' in input_source or '\\' in input_source or input_source.endswith(('.py', '.ipynb')):
        if input_source.endswith('.py'):
            return InputType.PYTHON_FILE
        elif input_source.endswith('.ipynb'):
            return InputType.NOTEBOOK
    
    # Default to code string
    return InputType.CODE

In [ ]:
#| export
def scan_input(
    input_source: str,  # Code string, Python file path, or notebook path
    input_type: Optional[InputType] = None  # Optional explicit input type. If None, will auto-detect.
) -> List[ClsPattern]:  # List of ClsPattern objects found
    "Scan various input types for cls= patterns."
    # Auto-detect input type if not specified
    if input_type is None:
        input_type = detect_input_type(input_source)
    
    # Scan based on input type
    if input_type == InputType.CODE:
        return scan_python_code(input_source)
    elif input_type == InputType.PYTHON_FILE:
        return scan_python_file(input_source)
    elif input_type == InputType.NOTEBOOK:
        return scan_jupyter_notebook(input_source)
    else:
        print(f"Error: Unknown input type: {input_type}")
        return []

In [ ]:
#| export
def analyze_input(
    input_source: str,  # Code string, Python file path, or notebook path
    input_type: Optional[InputType] = None  # Optional explicit input type. If None, will auto-detect.
) -> Dict[str, Any]:  # Dictionary with analysis results
    "Analyze any input type for replaceable CSS patterns."
    # Scan for patterns
    patterns = scan_input(input_source, input_type)
    
    # Get unique CSS classes
    unique_classes = get_unique_css_classes(patterns)
    
    # Get available classes from library
    assertion_patterns = collect_all_assertion_patterns()
    available_classes = get_available_css_classes(assertion_patterns)
    
    # Match CSS classes
    matches = match_css_classes(list(unique_classes), available_classes)
    
    # Detect actual input type used
    actual_input_type = input_type or detect_input_type(input_source)
    
    # Prepare results
    results = {
        'input_type': actual_input_type.value,
        'input_source': input_source if actual_input_type == InputType.CODE else os.path.basename(input_source),
        'total_patterns': len(patterns),
        'unique_classes': len(unique_classes),
        'matches': matches,
        'summary': {
            'exact_matches': sum(1 for m in matches.values() if m.match_type == MatchType.EXACT),
            'pattern_matches': sum(1 for m in matches.values() if m.match_type == MatchType.PATTERN),
            'no_matches': sum(1 for m in matches.values() if m.match_type == MatchType.NO_MATCH),
        },
        'patterns': patterns
    }
    
    return results

In [ ]:
#| export
def display_input_analysis(
    input_source: str,  # Code string, Python file path, or notebook path
    input_type: Optional[InputType] = None  # Optional explicit input type. If None, will auto-detect.
) -> None:  # TODO: Add return description
    "Analyze and display replaceable patterns from any input type."
    results = analyze_input(input_source, input_type)
    
    print("Pattern Analysis Report")
    print("=" * 60)
    print(f"Input Type: {results['input_type']}")
    print(f"Source: {results['input_source'][:50]}..." if results['input_type'] == 'code' else f"File: {results['input_source']}")
    print(f"Total cls= patterns found: {results['total_patterns']}")
    print(f"Unique CSS classes: {results['unique_classes']}")
    print()
    
    # Display summary
    summary = results['summary']
    total_replaceable = summary['exact_matches'] + summary['pattern_matches']
    print(f"Replaceable Classes: {total_replaceable}/{results['unique_classes']}")
    print(f"  - Exact matches: {summary['exact_matches']}")
    print(f"  - Pattern matches: {summary['pattern_matches']}")
    print(f"  - No matches: {summary['no_matches']}")
    print()
    
    # Display matches by type
    display_match_results(results['matches'])
    
    # Show patterns with line numbers
    if results['patterns']:
        print("\nPatterns by Location:")
        print("-" * 60)
        for pattern in results['patterns']:
            replaceable = "✓" if pattern.css_classes and any(
                results['matches'].get(cls, CSSClassMatch(cls, MatchType.NO_MATCH)).match_type != MatchType.NO_MATCH 
                for cls in pattern.css_classes
            ) else "✗"
            
            print(f"Line {pattern.line_number}: {replaceable} {pattern.full_expression}")
            if pattern.uses_combine_classes:
                print(f"         ↳ Already uses combine_classes")

In [ ]:
#| export
def analyze_and_suggest_input(
    input_source: str,  # Code string, Python file path, or notebook path
    input_type: Optional[InputType] = None  # Optional explicit input type. If None, will auto-detect.
) -> None:  # TODO: Add return description
    "Perform complete analysis with migration suggestions for any input type."
    # First show the analysis
    display_input_analysis(input_source, input_type)
    
    # Get analysis results
    results = analyze_input(input_source, input_type)
    summary = results['summary']
    
    if summary['exact_matches'] > 0 or summary['pattern_matches'] > 0:
        print("\n")
        
        # Get assertion patterns for suggestions
        assertion_patterns = collect_all_assertion_patterns()
        
        # Get migration suggestions
        suggestions = get_migration_suggestions(results['matches'], assertion_patterns)
        
        if suggestions:
            print("Migration Suggestions")
            print("=" * 60)
            print()
            
            for css_class in sorted(suggestions.keys()):
                match = results['matches'][css_class]
                match_type = "✓" if match.match_type == MatchType.EXACT else "~"
                
                print(f"{match_type} {css_class}:")
                for suggestion in suggestions[css_class]:
                    print(f"  → {suggestion}")
                print()

## Testing File Input Support

Test the unified interface with different input types:

In [ ]:
# Test auto-detection of input types
test_inputs = [
    'print("hello")',  # Code string
    'example.py',      # Python file
    'notebook.ipynb',  # Jupyter notebook
    '/path/to/file.py',  # Full path
    'Some("code", cls="flex")'  # Code with cls
]

print("Testing input type detection:")
for inp in test_inputs:
    detected = detect_input_type(inp)
    print(f"  '{inp}' → {detected.value}")

Testing input type detection:
  'print("hello")' → code
  'example.py' → python_file
  'notebook.ipynb' → notebook
  '/path/to/file.py' → python_file
  'Some("code", cls="flex")' → code


In [ ]:
# Test unified scanning interface with code string
print("Testing unified scan_input with code string:")
print("=" * 60)

test_code_unified = '''
from fasthtml.common import Div, Card

card = Card(
    Div("Title", cls="text-2xl font-semibold mb-4"),
    Div("Content", cls="text-gray-700"),
    cls="p-6 bg-white rounded-lg shadow-md"
)
'''

patterns = scan_input(test_code_unified)
print(f"Found {len(patterns)} patterns")
for pattern in patterns:
    print(f"  Line {pattern.line_number}: {pattern.css_classes}")

Testing unified scan_input with code string:
Found 3 patterns
  Line 7: ['p-6', 'bg-white', 'rounded-lg', 'shadow-md']
  Line 5: ['text-2xl', 'font-semibold', 'mb-4']
  Line 6: ['text-gray-700']


In [ ]:
# Create a temporary Python file for testing
import tempfile

test_py_content = '''
from fasthtml.common import *

def create_navbar():
    return Nav(
        Div("Logo", cls="font-bold text-xl"),
        Ul(
            Li("Home", cls="px-4 py-2"),
            Li("About", cls="px-4 py-2"),
            Li("Contact", cls="px-4 py-2"),
            cls="flex gap-4"
        ),
        cls="flex justify-between items-center p-4 bg-gray-100"
    )
'''

# Write to temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(test_py_content)
    temp_py_file = f.name

print(f"Testing Python file scanning:")
print(f"Temporary file: {temp_py_file}")
print("=" * 60)

# Scan the Python file
patterns = scan_input(temp_py_file)
print(f"Found {len(patterns)} patterns")
for pattern in patterns:
    print(f"  Line {pattern.line_number}: {pattern.css_classes}")

# Clean up
import os
os.unlink(temp_py_file)

Testing Python file scanning:
Temporary file: /tmp/tmp41mrapz5.py
Found 6 patterns
  Line 13: ['flex', 'justify-between', 'items-center', 'p-4', 'bg-gray-100']
  Line 6: ['font-bold', 'text-xl']
  Line 11: ['flex', 'gap-4']
  Line 8: ['px-4', 'py-2']
  Line 9: ['px-4', 'py-2']
  Line 10: ['px-4', 'py-2']


In [ ]:
# Test complete analysis with code string
print("\nTesting complete analysis with display_input_analysis:")
print()
display_input_analysis(test_code)


Testing complete analysis with display_input_analysis:

Pattern Analysis Report
Input Type: code
Source: 
from fasthtml.common import Div, Header, Nav, Mai...
Total cls= patterns found: 3
Unique CSS classes: 5

Replaceable Classes: 5/5
  - Exact matches: 5
  - Pattern matches: 0
  - No matches: 0

CSS Class Analysis Results:

✓ Exact Matches (5):
  - flex
  - font-bold
  - items-center
  - px-6
  - py-4

Summary: 5/5 classes are potentially replaceable

Patterns by Location:
------------------------------------------------------------
Line 12: ✓ combine_classes('flex', justify.between, items.center, 'px-6', 'py-4', 'items-center')
         ↳ Already uses combine_classes
Line 6: ✓ 'font-bold'
Line 9: ✓ combine_classes('flex', gap(4))
         ↳ Already uses combine_classes


In [ ]:
# Test with suggestions
print("\nTesting analysis with suggestions:")
print()
analyze_and_suggest_input(test_code_unified)


Testing analysis with suggestions:

Pattern Analysis Report
Input Type: code
Source: 
from fasthtml.common import Div, Card

card = Car...
Total cls= patterns found: 3
Unique CSS classes: 8

Replaceable Classes: 8/8
  - Exact matches: 6
  - Pattern matches: 2
  - No matches: 0

CSS Class Analysis Results:

✓ Exact Matches (6):
  - bg-white
  - font-semibold
  - mb-4
  - rounded-lg
  - shadow-md
  - text-2xl

~ Pattern Matches (2):
  - p-6 → matches pattern 'p-*'
    Examples: p-0, p-2.5, p-4
  - text-gray-700 → matches pattern 'text-gray-*'
    Examples: text-gray-500

Summary: 8/8 classes are potentially replaceable

Patterns by Location:
------------------------------------------------------------
Line 7: ✓ 'p-6 bg-white rounded-lg shadow-md'
Line 5: ✓ 'text-2xl font-semibold mb-4'
Line 6: ✓ 'text-gray-700'


Migration Suggestions

✓ bg-white:
  → View example: cjm-tailwind-explore example backgrounds color

✓ font-semibold:
  → View example: cjm-tailwind-explore example typography 

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()